<a href="https://colab.research.google.com/github/Shawon1024/ShawOnline/blob/main/Text_File_to_CSV_Convert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import pandas as pd
import numpy as np
from glob import glob
import re
import os

In [21]:
dataset_dir = '/content/drive/My Drive/Colab Notebooks/HARDATA/'

In [38]:
def extract_ids(filename):
    match = re.search(r'exp(\d+)_user(\d+)', filename)
    if match:
        return map(int, match.groups())
    else:
        raise ValueError(f"Filename {filename} does not match expected pattern.")

In [40]:
activity_labels_path = os.path.join(dataset_dir, 'activity_labels.txt')
activity_labels_df = pd.read_csv(activity_labels_path, header=None, sep=r'\s+', engine='python', names=['id', 'activity'])
activity_labels_dict = dict(zip(activity_labels_df.id, activity_labels_df.activity))

In [41]:
features_path = os.path.join(dataset_dir, 'features.txt')
features_df = pd.read_csv(features_path, header=None, names=['feature_names'])

In [42]:
labels_path = os.path.join(dataset_dir, 'labels.txt')
labels_df = pd.read_csv(labels_path, header=None, sep=' ', names=['experiment', 'user', 'activity_id', 'start', 'end'])

In [43]:
merged_data = pd.DataFrame()

In [44]:
import glob

In [45]:
acc_files = glob.glob(os.path.join(dataset_dir, 'acc_exp*_user*.txt'))
gyro_files = glob.glob(os.path.join(dataset_dir, 'gyro_exp*_user*.txt'))

In [48]:
for acc_file, gyro_file in zip(sorted(acc_files), sorted(gyro_files)):
    match_acc = re.search(r'exp(\d+)_user(\d+)', acc_file)
    if match_acc:
        exp_id, user_id = map(int, match_acc.groups())
    else:
        raise ValueError(f"Filename {acc_file} does not match expected pattern.")

    activity_id = labels_df[(labels_df.experiment == exp_id) & (labels_df.user == user_id)]['activity_id'].iloc[0]
    activity_name = activity_labels_dict[activity_id]

In [49]:
gyro_data = pd.read_csv(gyro_file, sep=' ', header=None, engine='python')
acc_data = pd.read_csv(acc_file, sep=' ', header=None, engine='python')

In [50]:
combined_data = pd.concat([acc_data, gyro_data], axis=1)
combined_data['activity'] = activity_name

In [51]:
merged_data = pd.concat([merged_data, combined_data], ignore_index=True)

In [53]:
feature_names = features_df['feature_names'].tolist() * 2
feature_names.append('activity')
merged_data.colunms = feature_names[:len(merged_data.columns)]

<ipython-input-53-c7cda795549d>:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  merged_data.colunms = feature_names[:len(merged_data.columns)]


In [54]:
output_file = os.path.join(dataset_dir, 'Sensor_Data.csv')
merged_data.to_csv(output_file, index=False)
print(f"Merged Data Saved to {output_file}")

Merged Data Saved to /content/drive/My Drive/Colab Notebooks/HARDATA/Sensor_Data.csv
